## Gradient Boosting

# 1. Load Data and Import Libraries/Functions

In [ ]:
import numpy as np
import pandas as pd
import pickle
import cupy
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error, accuracy_score
from xgboost import XGBRegressor, XGBClassifier
from utilities import cross_val_metrics_calculate

In [ ]:
data = pd.read_csv("../data/train_data_2nd.csv")
X = data.iloc[:, 1:-1]
y = data.iloc[:, -1]

In [27]:
# data for property type classification
y = data['Property Type']
X = data.loc[:, data.columns != 'Property Type'].iloc[:, 1:]

In [28]:
feature_names = X.columns
feature_names

Index(['Area (m2)', 'Bedrooms', 'Bathrooms', 'Address', 'Law Document',
       'Quarter', 'Year', 'Latitude', 'Longitude', 'Postal Code', 'Importance',
       'Place Rank', 'City', 'Price (billion VND)'],
      dtype='object')

In [8]:
n_folds = 5
kfold = KFold(n_folds)

# 2. Sklearn's Gradient Boosting

In [ ]:
gb_search = GridSearchCV(GradientBoostingRegressor(),
                         param_grid={'loss':['absolute_error'],
                                     'n_estimators':[100,200,250,300],
                                     'learning_rate':[0.05, 0.1, 0.25, 0.5],
                                     'max_depth':[3,5,7],
                                     'max_features':['sqrt','log2'],
                                     'n_iter_no_change':[3]},
                         scoring=['neg_mean_absolute_error',
                                  'neg_mean_absolute_percentage_error'],
                         cv=5,
                         refit='neg_mean_absolute_percentage_error')

gb_search.fit(X, y)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.05, 0.1, 0.25, 0.5],
                         'loss': ['absolute_error'], 'max_depth': [3, 5, 7],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100, 200, 250, 300],
                         'n_iter_no_change': [3]},
             refit='neg_mean_absolute_percentage_error',
             scoring=['neg_mean_absolute_error',
                      'neg_mean_absolute_percentage_error'])

In [ ]:
gb_search.best_params_

{'learning_rate': 0.05,
 'loss': 'absolute_error',
 'max_depth': 3,
 'max_features': 'log2',
 'n_estimators': 100,
 'n_iter_no_change': 3}

In [ ]:
# gb = gb_search.best_estimator_
gb = GradientBoostingRegressor(learning_rate=0.05, loss='absolute_error', max_depth=3, max_features='log2', n_estimators=100, n_iter_no_change=3)

cv_results = cross_val_metrics_calculate(gb, X, y, kfold.split(X))
print(cv_results)

{'mse': 568.9951529084808, 'rmse': 23.724508361992804, 'mae': 5.939081113789564, 'mape': 1.9131176489511437, 'medae': 1.5926783284779256, 'medape': 0.32133772163249547}


In [ ]:
gb.fit(X, y)
for i in range(gb.n_features_in_):
  print(f"{feature_names[i]}: {gb.feature_importances_[i]}")

Area (m2): 0.30331960369761424
Property Type: 0.21009416117675786
Bedrooms: 0.13014135806985908
Bathrooms: 0.06297712696430681
Address: 0.0076915662332285376
Law Document: 0.026722580068140916
Quarter: 0.005081212291055695
Year: 0.07916393886496169
Latitude: 0.054384188386333264
Longitude: 0.05450221042460891
Postal Code: 0.04130066016751146
Importance: 0.0025238516770344343
Place Rank: 0.0015183997968915312
City: 0.020579142181695587


- Compared to SVM: higher MAPE (1.95 vs 1.58) but lower MAE (5.85 vs 7) -> generalize better between lower-priced and higher-priced estates
- Area and property type has high importances, along with bedrooms and bathrooms num.

In [ ]:
pickle.dump(gb, open("../models/GradientBoosting.h5", 'wb'))

**With standardization**

In [ ]:
gb_search_s = make_pipeline(StandardScaler(), gb_search)

gb_search_s.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gridsearchcv',
                 GridSearchCV(cv=5, estimator=GradientBoostingRegressor(),
                              param_grid={'learning_rate': [0.05, 0.1, 0.25,
                                                            0.5],
                                          'loss': ['absolute_error'],
                                          'max_depth': [3, 5, 7],
                                          'max_features': ['sqrt', 'log2'],
                                          'n_estimators': [100, 200, 250, 300],
                                          'n_iter_no_change': [3]},
                              refit='neg_mean_absolute_percentage_error',
                              scoring=['neg_mean_absolute_error',
                                       'neg_mean_absolute_percentage_error']))])

In [ ]:
gb_search_s.named_steps['gridsearchcv'].best_params_

{'learning_rate': 0.05,
 'loss': 'absolute_error',
 'max_depth': 3,
 'max_features': 'log2',
 'n_estimators': 100,
 'n_iter_no_change': 3}

In [ ]:
gb_with_standardize = make_pipeline(
    StandardScaler(),
    GradientBoostingRegressor(
        learning_rate=0.05, loss='absolute_error',
        max_depth=3, max_features='log2', n_estimators=100, n_iter_no_change=3
    )
)

cv_results = cross_val_metrics_calculate(gb_with_standardize, X, y, kfold.split(X))
print(cv_results)

{'mse': 569.4697579136764, 'rmse': 23.737490446174988, 'mae': 5.954853194239254, 'mape': 1.9029533236992393, 'medae': 1.5964026935868727, 'medape': 0.3216995441758531}


In [ ]:
gb_with_standardize.fit(X, y)
for i in range(gb_with_standardize.n_features_in_):
  print(f"{feature_names[i]}: {gb_with_standardize.named_steps['gradientboostingregressor'].feature_importances_[i]}")

Area (m2): 0.3140035352413854
Property Type: 0.2193541792423939
Bedrooms: 0.11206848355916987
Bathrooms: 0.07825515775472998
Address: 0.007158923518480857
Law Document: 0.020646687303835013
Quarter: 0.004828982487499203
Year: 0.07389214458532888
Latitude: 0.04929548667845499
Longitude: 0.05713444064419809
Postal Code: 0.03770889535046663
Importance: 0.0019639210987666975
Place Rank: 0.0014674776288361008
City: 0.022221684906454136


With standardized data: no difference

In [ ]:
pickle.dump(gb_with_standardize, open("../models/GradientBoosting.h5", 'wb'))

# 3. XGBoost's Gradient Boosting

In [ ]:
xgb_search = GridSearchCV(
    XGBRegressor(
        booster='gbtree',
        tree_method='hist',
        eval_metric=mean_absolute_percentage_error,
        device='cuda'
        ),
    param_grid={
        'n_estimators':[300,400,500],
        'learning_rate':[0.01, 0.025, 0.05, 0.1, 0.25],
        'max_depth':[7,10,12]
        },
    scoring=['neg_mean_absolute_error',
             'neg_mean_absolute_percentage_error'],
    cv=5,
    refit='neg_mean_absolute_percentage_error'
    )

xgb_search.fit(cupy.array(X), y)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster='gbtree',
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device='cuda',
                                    early_stopping_rounds=None,
                                    enable_categorical=False,
                                    eval_metric=<function mean_absolute_percentage_error at 0x7e382d921ea0>,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_t...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.025, 0.05, 0.1, 0.25],
                         'max_depth': [7, 10, 12],
                         'n_estimators': [300, 400, 500]},
             refit='neg_mean_absolute_percentage_error',
             scoring=['neg_mean_absolute_error',
                      'neg_mean_absolute_percentage_error'])

In [ ]:
# xgb = xgb_search.best_estimator_
xgb = XGBRegressor(
    booster='gbtree',
    tree_method='hist',
    eval_metric=mean_absolute_percentage_error,
    device='cuda',
    learning_rate=0.05,
    max_depth=12,
    n_estimators=500
)

cv_results = cross_val_metrics_calculate(xgb, X, y, kfold.split(X))
print(cv_results)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:48:01] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


{'mse': 495.84911628314114, 'rmse': 21.925027608065587, 'mae': 5.186057149184199, 'mape': 2.497323612739722, 'medae': 1.2785976581573486, 'medape': 0.23926301702752123}


In [ ]:
xgb.fit(X, y)

for i in range(xgb.n_features_in_):
  print(f"{feature_names[i]}: {xgb.feature_importances_[i]}")

Area (m2): 0.027917200699448586
Property Type: 0.043519891798496246
Bedrooms: 0.02664334326982498
Bathrooms: 0.023635923862457275
Address: 0.01623951643705368
Law Document: 0.022283319383859634
Quarter: 0.010827318765223026
Year: 0.026958506554365158
Latitude: 0.027839701622724533
Longitude: 0.04177043214440346
Postal Code: 0.0360429473221302
Importance: 0.05759327486157417
Place Rank: 0.03564010187983513
City: 0.6030884981155396


- Lower RMSE and MAE, but higher MAPE (1.90 -> 2.497); maybe acceptable as MAPE does not increase too much?
- City: 0.603 -> heavily dependent on city
- Importance, property type, longitude

In [ ]:
xgb.save_model("../models/XGBoostRegressor.json")

**With standardization**

In [ ]:
xgb_search = GridSearchCV(
    XGBRegressor(
        booster='gbtree',
        tree_method='hist',
        eval_metric=mean_absolute_percentage_error,
        device='cuda'
        ),
    param_grid={
        'n_estimators':[300,400,500],
        'learning_rate':[0.01, 0.025, 0.05, 0.1, 0.25],
        'max_depth':[7,10,12]
        },
    scoring=['neg_mean_absolute_error',
             'neg_mean_absolute_percentage_error'],
    cv=5,
    refit='neg_mean_absolute_percentage_error'
    )
# xgb_search_s = make_pipeline(StandardScaler(), xgb_search)

xgb_search.fit(cupy.array(StandardScaler().fit_transform(X)), y)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster='gbtree',
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device='cuda',
                                    early_stopping_rounds=None,
                                    enable_categorical=False,
                                    eval_metric=<function mean_absolute_percentage_error at 0x7e382d921ea0>,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_t...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.025, 0.05, 0.1, 0.25],
                         'max_depth': [7, 10, 12],
                         'n_estimators': [300, 400, 500]},
             refit='neg_mean_absolute_percentage_error',
             scoring=['neg_mean_absolute_error',
                      'neg_mean_absolute_percentage_error'])

In [ ]:
xgb_with_standardize = make_pipeline(
    StandardScaler(),
    XGBRegressor(
        booster='gbtree',
        tree_method='hist',
        eval_metric=mean_absolute_percentage_error,
        device='cuda',
        learning_rate=0.1,
        max_depth=12,
        n_estimators=400
    )
)

cv_results = cross_val_metrics_calculate(xgb_with_standardize, X, y, kfold.split(X))
print(cv_results)

{'mse': 497.03853084874135, 'rmse': 21.945368255170266, 'mae': 5.229652894139923, 'mape': 2.504219923516407, 'medae': 1.3255969047546388, 'medape': 0.24285307204352188}


In [ ]:
xgb_with_standardize.fit(X, y)

for i in range(xgb_with_standardize.named_steps['xgbregressor'].n_features_in_):
  print(f"{feature_names[i]}: {xgb_with_standardize.named_steps['xgbregressor'].feature_importances_[i]}")

Area (m2): 0.020956967025995255
Property Type: 0.03202591836452484
Bedrooms: 0.02316567301750183
Bathrooms: 0.02074238657951355
Address: 0.012834804132580757
Law Document: 0.018069058656692505
Quarter: 0.010597722604870796
Year: 0.02209646999835968
Latitude: 0.02232278510928154
Longitude: 0.03491033613681793
Postal Code: 0.030435485765337944
Importance: 0.049392905086278915
Place Rank: 0.0665590912103653
City: 0.6358904242515564


- Little difference in feature importance compared to non-standardized data; area has less importance as it is converted into a smaller range -> smaller difference between estates -> harder to classify
- Cross-validation results: slightly worse

## Gradient Boosting Classifier

# 1. Sklearn

In [ ]:
gb_search = GridSearchCV(
    HistGradientBoostingClassifier(),
    param_grid={
        # 'n_estimators':[100,200,250,300],
        'learning_rate':[0.001, 0.0025, 0.005, 0.01, 0.025, 0.05, 0.1],
        'max_depth':[3,5,7,10],
        'max_features':[0.15, 0.2, 0.3],
    },
    scoring=[
        'accuracy'
    ],
    cv=5,
    refit='accuracy'
)

# gb_search.fit(X, y)

In [ ]:
gb = gb_search.best_estimator_
# gb = HistGradientBoostingClassifier(
#     learning_rate=0.05,
#     max_depth=10,
#     max_features=0.3
# )

cv_results = cross_val_metrics_calculate(gb, X, y, kfold.split(X), metrics=['accuracy','precision','recall','f1'])
print(cv_results)

{'accuracy': 0.8158659583813177, 'precision': 0.7454468260865551, 'recall': 0.7247844463247505, 'f1': 0.7333102836790409}


In [ ]:
pickle.dump(gb, open("../models/GradientBoosting_Classifier.h5", 'wb'))

In [ ]:
gb_with_standardize = make_pipeline(
    StandardScaler(),
    HistGradientBoostingClassifier(
        learning_rate=0.05,
        max_depth=10,
        max_features=0.3
    )
)

cv_results = cross_val_metrics_calculate(gb_with_standardize, X, y, kfold.split(X), metrics=['accuracy','precision','recall','f1'])
print(cv_results)

{'accuracy': 0.8141700075307577, 'precision': 0.7208556908911417, 'recall': 0.7504340821907887, 'f1': 0.7242958962645355}


- Lower precision, higher recall -> less precise predictions

# 2. XGBoost Classifier

In [ ]:
xgb_search = GridSearchCV(
    XGBClassifier(
        booster='gbtree',
        tree_method='hist',
        eval_metric=accuracy_score,
        device='cuda'
    ),
    param_grid={
        'n_estimators':[250,300,400,500],
        'learning_rate':[0.01, 0.025, 0.05, 0.1, 0.25],
        'max_depth':[7,10,12]
    },
    scoring=['accuracy'],
    cv=5,
    refit='accuracy'
)

xgb_search.fit(cupy.array(X), y)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster='gbtree',
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric=<function accuracy_score at 0x7ba7d6972d40>,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     intera...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.025, 0.05, 0.1, 0.25],
                         'max_depth': [7, 10, 12],
                         'n_estimators': [250, 300, 400, 500]},
             refit='accuracy', scoring=['accuracy'])

In [29]:
# xgb = xgb_search.best_estimator_
xgb = XGBClassifier(
    booster='gbtree',
    tree_method='hist',
    eval_metric=accuracy_score,
    device='cpu',
    learning_rate=0.1,
    max_depth=7,
    n_estimators=300
)

cv_results = cross_val_metrics_calculate(xgb, X, y, kfold.split(X), metrics=['accuracy','precision','recall','f1'])
print(cv_results)

{'accuracy': 0.8314411623086443, 'precision': 0.7942497971655549, 'recall': 0.7847648578401611, 'f1': 0.7866994682641925}


- Better results in all metrics
- Tree-based methods usually do not work well with standardized data -> skip

In [19]:
xgb.fit(X, y)

for i in range(xgb.n_features_in_):
  print(f"{feature_names[i]}: {xgb.feature_importances_[i]}") # without feature selection

Area (m2): 0.06314612179994583
Bedrooms: 0.13606585562229156
Bathrooms: 0.07565029710531235
Address: 0.0140488026663661
Law Document: 0.06905309110879898
Quarter: 0.022826069965958595
Year: 0.04927624389529228
Latitude: 0.024088313803076744
Longitude: 0.03709310665726662
Postal Code: 0.018648166209459305
Importance: 0.013848182745277882
Place Rank: 0.025391940027475357
City: 0.40668371319770813
Price (billion VND): 0.04418011009693146


**With feature selection: Remove address, importance, quarter**

In [20]:
# Feature selection
X = X.loc[:, ["Area (m2)", "Bedrooms", "Bathrooms", "Law Document",
              "Year", "Latitude", "Longitude", "Place Rank", "City", "Price (billion VND)"]]

In [21]:
feature_names = X.columns
feature_names

Index(['Area (m2)', 'Bedrooms', 'Bathrooms', 'Law Document', 'Year',
       'Latitude', 'Longitude', 'Place Rank', 'City', 'Price (billion VND)'],
      dtype='object')

In [24]:
xgb_search = GridSearchCV(
    XGBClassifier(
        booster='gbtree',
        tree_method='hist',
        eval_metric=accuracy_score,
        device='cuda'
    ),
    param_grid={
        'n_estimators':[250,300,400,500],
        'learning_rate':[0.01, 0.025, 0.05, 0.1, 0.25],
        'max_depth':[7,10,12]
    },
    scoring=['accuracy'],
    cv=5,
    refit='accuracy'
)

xgb_search.fit(cupy.array(X), y)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster='gbtree',
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric=<function accuracy_score at 0x78a888d17d90>,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     intera...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.025, 0.05, 0.1, 0.25],
                         'max_depth': [7, 10, 12],
                         'n_estimators': [250, 300, 400, 500]},
             refit='accuracy', scoring=['accuracy'])

In [25]:
# xgb = xgb_search.best_estimator_
xgb = XGBClassifier(
    booster='gbtree',
    tree_method='hist',
    eval_metric=accuracy_score,
    device='cpu',
    learning_rate=0.1,
    max_depth=7,
    n_estimators=400
)

cv_results = cross_val_metrics_calculate(xgb, X, y, kfold.split(X), metrics=['accuracy','precision','recall','f1'])
print(cv_results)

{'accuracy': 0.8204001298311507, 'precision': 0.776270421112055, 'recall': 0.7640156151130688, 'f1': 0.7654537675446356}


In [26]:
xgb.fit(X, y)

for i in range(xgb.n_features_in_):
  print(f"{feature_names[i]}: {xgb.feature_importances_[i]}") # with feature selection

Area (m2): 0.061008989810943604
Bedrooms: 0.12795118987560272
Bathrooms: 0.07327907532453537
Law Document: 0.0677880272269249
Year: 0.04600735381245613
Latitude: 0.0252000093460083
Longitude: 0.03523192182183266
Place Rank: 0.021918663755059242
City: 0.4974979758262634
Price (billion VND): 0.04411673545837402


- Not better than without selection

In [ ]:
xgb.fit(cupy.array(X), y)
xgb.set_params(device='cpu')
xgb.save_model("../models/XGBoostClassifier.json")